## Network Creater: Larger DEMs

This notebook tests the network model grid creator on a DEM of the Skokomish river basin in Western Washington with (416 x 320) 100m grid cells

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os

# landlab modules
from landlab.plot.graph import plot_nodes, plot_links
from landlab.io import read_esri_ascii, write_esri_ascii

# Package for plotting raster data
from landlab.plot.imshow import imshow_grid
from landlab.grid.create_network import network_grid_from_raster

from landlab.grid.create_network import (
    AtMostNodes,
    JustEndNodes,
    SpacingAtLeast,
    spacing_from_drainage_area,
)

### Load DEM as a RasterModelGrid

local directory for DEM (eventually upload to folder so this is not necessary)

In [ ]:
DEM_dir = r"C:\Users\sahrendt\OneDrive\525 Tectonic Geomorphology\Final_Proposal\data"

read file

In [ ]:
grid, z = read_esri_ascii(
    os.path.join(DEM_dir, "skokomish_elev_100m_gdalwarp.asc"),
    name="topographic__elevation"
)
grid.status_at_node[np.ma.masked_where(z == -9999, z).mask] = grid.BC_NODE_IS_CLOSED 

### Set a watershed boundary condition for your grid

This function uses the minimum elevation node nearest to the nodata values. This DEM only has one possible node, so it is easy for the function to determine. If an error is thrown that says 'Grid has multiple potential outlet nodes' choose one from the list of node IDs shown in the error window. (You may want to plot them on your DEM to verify that they make sense as an outlet node--see below). This error may happen when there are a few very low elevation cells.
* Note: this step is important for deliniating the channel. If you find the code is taking a long time to generate the NetworkModelGrid, check which node you set as the boundary condition.

In [ ]:
grid.set_watershed_boundary_condition(
    'topographic__elevation',
    nodata_value=-9999.0,
    return_outlet_id=True
)

### Plot the DEM and outlet node

In [ ]:
imshow_grid(
    grid,
    "topographic__elevation",
    plot_name="Basin topography",
    color_for_closed=None,
    colorbar_label="$z$ [m]",
    vmin=0
)
#here you can test your outlet node location
#looks like what we'd expect for the outlet node here
test_outlet_node_id = 25900
plt.scatter(
    grid.node_x[test_outlet_node_id],
    grid.node_y[test_outlet_node_id],
    color='red'
)

### Generate Network

Now we will generate a network using the default placement of a NetworkModelGrid node for every corresponding RasterModelGrid node. You may want to coarsen this later when you decide what is an acceptable NetworkModelGrid spacing for a NetworkSedimentTransporter model. (i.e. stability). Some notes:

* Be SURE you have a ballpark minimum channel threshhold: this is a drainage area threshhold that sets the upstream drainage area for which you want to truncate your channel network. Start high, and then reduce for a more complex network that extends to higher elevations.
* Be sure you have your outlet node defined
* You can pass information from the RasterModelGrid nodes to the NetworkModelGrid nodes using the 'include' variable if you have other things attached to your RasterModelGrid that would be helpful for the NetworkModelGrid to know (i.e. slope)

In [ ]:
network_grid = network_grid_from_raster(
    grid,
    minimum_channel_threshold=25000000,
    include=["drainage_area", "topographic__elevation"],
)

In [ ]:
imshow_grid(
    grid,
    "topographic__elevation",
    plot_name="Basin topography",
    color_for_closed=None,
    colorbar_label="$z$ [m]",
)
plot_links(network_grid, with_id=False)
plot_nodes(network_grid, with_id=False, markersize=0.2)

### Refine Network using Reducers

Now we will use the Reducer options to reduce the number of nodes on the network. This shows an example where 10 nodes are placed on each segment of the river between bifurcations.

In [ ]:
network_grid = network_grid_from_raster(
    grid,
    reducer=AtMostNodes(count=10),
    minimum_channel_threshold=25000000,
    include=["drainage_area", "topographic__elevation"],
)

plot_nodes(network_grid, markersize=4)
plot_links(network_grid)
plt.title("Nodes and Links");

Realistically, you may want to reduce your NetworkModelGrid node spacing in the steeper, higher elevation areas of the basin for stability of the NetworkSedimentTransporter model. Here, we show an option to define the network node spacing from an upstream drainage area relation that estimates river width from the Frasson et al. 2019 (GRL) power law relationship:

$W=a*DA^b$

where $W$ is width $DA$ is drainage area and $a$ and $b$ are parameters. The function uses default values for a and b reported in the paper from fitting field data, but you  may want to adjust this depending on your river system. Changing n_widths will adjust node spacing by a certain multiple of local river width.

In [ ]:
spacing = spacing_from_drainage_area(
    grid.at_node["drainage_area"],
    a=9.68,
    b=0.32,
    n_widths=50
)


network_grid = network_grid_from_raster(
    grid,
    reducer=SpacingAtLeast(grid.xy_of_node, spacing),
    minimum_channel_threshold=50000000,
    include=["drainage_area", "topographic__elevation"],
)

plot_nodes(network_grid, with_id=False, markersize=4)
plot_links(network_grid, with_id=False)
plt.title("Nodes and Links");